### 1st model attempt###
First we import all the relevant libraries

In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils import data
import torch.optim as optim

The train_loader is used in order to group the data into batches.
Here, I am assuming the train_set contains only the 4-fold and complete MRI scans. We don't need the 8-fold yet. 

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32) #can also try 64 or 128

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.device_count() #check number of GPUs

Let's just take one batch from the train_loader and check if the model is working using it. 
I also made a grid to visualise the images which hopefully will work.

In [ ]:
batch = next(iter(train_loader))
four_folds, full_scans = batch[0].to(device), batch[1].to(device)
print(four_folds.shape)
print(full_scans.shape)
grid = torchvision.utils.make_grid(four_folds[:][0:7], nrow=2)
plt.figure(figsize=(15,15))
plt.imshow(np.transpose(grid, (1,2,0)))
plt.show()

This is the neural net architecture. The layers are the same as the original AlexNet. I have changed a lot of the hyperparameters inside the model the original values were tuned for 256x images and ours are 320x. 

I have also changed the number of neurons in the fully connected layers in order to get a 320x image as the output of the forward propagation. 

Finally, I have applied a sigmoid activation function on the last layer to make sure all values are between 0 and 1, then I multiplied that by 255 and changed the type to Int32. Essentially, I transformed all output values into pixel values. 

The numbers I have commented next to the layers are the lengths of the square matrix on that specific layer.

In [ ]:
class AlexNet(nn.Module):

    def __init__(self, num_classes=102400):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=9, stride=5, padding=2), #64/64
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=4, stride=2),  #31/31
            nn.Conv2d(64, 192, kernel_size=5, padding=2), #31/31
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2), #15/15
            nn.Conv2d(192, 384, kernel_size=3, padding=1), #15/15
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1), #15/15
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), #15/15
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2), #7/7
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 7 * 7, 32768),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(32768, 65536),
            nn.ReLU(inplace=True),
            nn.Linear(65536, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        x = nn.functional.sigmoid(x)
        x = x * 255
        x = x.type(torch.int32)
        return x

## Set this to true before strating the backprop!!!! #

As long as this is false, the gradients cannot be computed. It does make the forward prop a little bit faster so I set it to false initially.  

In [ ]:
torch.set_grad_enabled(False) #set to true before starting the training!!!!

Based on how long the next step takes to compute, we can get a rough idea of how long we'd have to wait for the training process. If it takes a few seconds, then the training will probably take a few hours, so we might want to look again at the architecture and decide if we wanna change something first. 

In [ ]:
network = AlexNet()
network.to(device) #move the model on the GPU
#network = nn.DataParallel(network) #if we have access to more than 1 GPU
output = network(four_folds)
output

Function for mean absolute error... couldn't find it already built in pytorch

In [ ]:
def mae(output, target):
    loss = torch.mean(abs(output - target))
    return loss

This should, in theory, return the loss for all images in the batch combined. I flattened the full_scans. I'm pretty sure it won't work. I'll have to look at the shape of the data to change it properly. 

In [ ]:
loss = mae(output, torch.flatten(full_scans, 1))
loss.item()

This will compute the gradients after backprop and return the shape of the gradient tensor for the first layer, which should be the same as the shape of the weight tensor for that layer. 

In [ ]:
loss.backward()
network.features[0].weight.grad.shape

This will update all the weights based on the previously computed gradients. The algorithm I used for optimisation, Adam, basically makes sure the model will converge towards a minimum faster.  

In [ ]:
optimizer = optim.Adam(network.parameters(), lr=0.03)
optimizer.step() 
loss.item()

If everything went well so far, we're gonna try going through the whole training set once.
The total loss should hopefully decrease from one batch to the next.

In [ ]:
total_loss = []
for batch in train_loader:
    four_folds, full_scans = batch[0].to(device), batch[1].to(device)     #take the X and y out of the batch
    full_scans = torch.flatten(full_scans, 1)    #flatten the full scans
    output = network(four_folds)       #feedforward
    loss = mae(output, full_scans)     #compute the loss
    optimizer.zero_grad()       #set current gradients to 0
    loss.backward()      #backpropagate
    optimizer.step()     #update the weights
    total_loss.append(loss.item())

If by some miracle we get all the way here in a reasonable amount of time, we can try running multiple epochs and seeing how low we can get the loss function.

In [ ]:
for epoch in range(10):
    total_loss = 0
    for batch in train_loader:
        four_folds, full_scans = batch[0].to(device), batch[1].to(device)     #take the X and y out of the batch
        full_scans = torch.flatten(full_scans, 1)    #flatten the full scans
        output = network(four_folds)       #feedforward
        loss = mae(output, full_scans)     #compute the loss
        optimizer.zero_grad()       #set current gradients to 0
        loss.backward()      #backpropagate
        optimizer.step()     #update the weights
        total_loss += loss.item()
    print(total_loss, "  ")